In [1]:
import os
# Set the latest version of Spark
spark_version = 'spark-3.5.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,108 kB]
Get:14 https://r2u.stat.illino

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the CSV from the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

# Show DataFrame
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [4]:
# 2. Create a temporary view of the DataFrame.

df.createOrReplaceTempView('home_sales')

In [ ]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)



In [5]:
# 5A. What is the average price (round avg(price)) for a four bedroom (where bedroom = 4) house sold per year(group and order by year)?

spark.sql("""SELECT
                    year(date) as year_sold,
                    round(avg(price), 2) as avg_price
              FROM
                    home_sales
              where
                    bedrooms = 4
              group by
                    year(date)
              order by
                    year(date) desc
              """).show(10)

+---------+---------+
|year_sold|avg_price|
+---------+---------+
|     2022|296363.88|
|     2021|301819.44|
|     2020|298353.78|
|     2019| 300263.7|
+---------+---------+



In [6]:
# 5B. What is the average price (round avg(price)) of a home for each year (Group and order by data_built) the home was built
# that have 3 bedrooms and 3 bathrooms (Where bedrooms = 3 and bathrooms = 3)?

spark.sql("""SELECT
                    date_built,
                    round(avg(price), 2) as avg_price
              FROM
                    home_sales
              where
                    bedrooms = 3
                and bathrooms = 3
              group by
                    date_built
              order by
                    date_built desc
              """).show(10)

+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2017|292676.79|
|      2016|290555.07|
|      2015| 288770.3|
|      2014|290852.27|
|      2013|295962.27|
|      2012|293683.19|
|      2011|291117.47|
|      2010|292859.62|
+----------+---------+



In [7]:
# 5C. What is the average price rounded to 2 decimal places (round avg(price)) of a home for each year the home was built (GROUP BY and ORDER BY date_built),
# that have 3 bedrooms, 3 bathrooms, with two floors
# and are greater than or equal to 2,000 square feet (Where bedrooms = 3 and bathrooms = 3 and floor = 2 and sqft_living >= 2000)?

spark.sql("""SELECT
                    date_built,
                    round(avg(price), 2) as avg_price
              FROM
                    home_sales
              where
                    bedrooms = 3
                and bathrooms = 3
                and floors = 2
                and sqft_living >= 2000
              group by
                    date_built
              order by
                    date_built desc
              """).show(10)

+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2017|280317.58|
|      2016| 293965.1|
|      2015|297609.97|
|      2014|298264.72|
|      2013|303676.79|
|      2012|307539.97|
|      2011|276553.81|
|      2010|285010.22|
+----------+---------+



In [8]:
# 5D. What is the average price of a home per "view" rating (group and order by view), rounded to two decimal places (price avg and rounded),
# having an average home price greater than or equal to $350,000 (having avg(price) > 350000)? Order by descending view rating (order by view desc).
# Although this is a small dataset, determine the run time for this query (runtime = 1.4545540809631348 seconds)?

start_time = time.time()

spark.sql("""SELECT
                    view,
                    round(avg(price), 2) as avg_price
              FROM
                    home_sales
              group by
                    view
              having
                    avg(price) >= 350000
              order by
                    view desc
              """).show(10)


print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
+----+----------+
only showing top 10 rows

--- 1.4545540809631348 seconds ---


In [9]:
# 6. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [10]:
# 7. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [11]:
# 8. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime (Runtime = 0.8161742687225342 seconds which is less than the uncached query runtime = 0.8161742687225342 seconds)

start_time = time.time()

spark.sql("""SELECT
                    view,
                    round(avg(price), 2) as avg_price
              FROM
                    home_sales
              group by
                    view
              having
                    avg(price) >= 350000
              order by
                    view desc
              """).show(10)

print("--- %s seconds ---" % (time.time() - start_time))


+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
+----+----------+
only showing top 10 rows

--- 0.8161742687225342 seconds ---


In [12]:
# 9. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")

In [14]:
# 10A. Read the parquet formatted data (partitioned by date_built)
parquet_df = spark.read.parquet('home_sales_partitioned')

In [15]:
# 10B. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView('home_sales_partitioned')

In [16]:
# 11. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime (runtime = 0.49514198303222656 seconds which is faster than the cached runtime = 0.8161742687225342 seconds)

start_time = time.time()

spark.sql("""SELECT
                    view,
                    round(avg(price), 2) as avg_price
              FROM
                    home_sales
              group by
                    view
              having
                    avg(price) >= 350000
              order by
                    view desc
              """).show(10)

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
+----+----------+
only showing top 10 rows

--- 0.49514198303222656 seconds ---


In [17]:
# EXTRA FROM CLASS Partition by the "view" field on the formatted parquet home sales data
df.write.partitionBy("view").mode("overwrite").parquet("home_sales_partitioned2")

In [20]:
# EXTRA FROM CLASS Create a temporary table for the parquet partitioned data by view
parquet_df_by_view = spark.read.parquet('home_sales_partitioned2')
parquet_df_by_view.createOrReplaceTempView('parquet_home_sales_by_view')

In [19]:
# EXTRA FROM CLASS Runtime is even faster (Took this extra step from PROF to see if we could create a view from the partitioned data to even get faster results)
start_time = time.time()

spark.sql("""SELECT
                    view,
                    round(avg(price), 2) as avg_price
              FROM
                    home_sales
              group by
                    view
              having
                    avg(price) >= 350000
              order by
                    view desc
              """).show(10)

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
+----+----------+
only showing top 10 rows

--- 0.47069501876831055 seconds ---


In [21]:
# 12. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [22]:
# 13. Check if the home_sales is no longer cached

spark.catalog.isCached("home_sales")

False